# Set file paths to Google Drive

In [1]:
pip install scipy==1.1.0

     |████████████████████████████████| 31.2 MB 25.2 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.4 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
plotnine 0.6.0 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
jax 0.2.25 requires scipy>=1.2.1, but you have scipy 1.1.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
#pip uninstall torch
!pip install torch==1.5.0
!pip install torchvision==0.6.0

     |████████████████████████████████| 752.0 MB 9.7 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.5.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.5.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.5.0 which is incompatible.
     |████████████████████████████████| 6.6 MB 6.0 MB/s 
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111


In [3]:
#pip install numpy==1.13 matplotlib==2.0 h5py==2.7 pathlib2==2.3 scipy==0.19.1 jupyter==1.0.0 pytorch==1.5.0 torchvision==0.2.1

In [3]:
import os
from google.colab import drive
import torch
import torchvision
import sys
import json
import h5py
import numpy as np
from scipy.misc import imread, imresize

drive.mount('/content/drive')
dataset_path = "/content/drive/MyDrive/Project_NeurosymbolicAI/6.Datasets/CLEVR_v1.0"
code_path = "/content/drive/MyDrive/Project_NeurosymbolicAI/7.Code/tbd-nets-encoder-main"
results_path = os.path.join(code_path, 'results')
data_path = os.path.join(code_path, 'data')
tbd_path = os.path.join(code_path, 'tbd')

### use module_net_back.py for original tbd
module_net_back = os.path.join(tbd_path,'module_net_back.py')

### use module_net.py for transformer modules
module_net = os.path.join(tbd_path,'module_net.py')
transformer = os.path.join(tbd_path,'transformer.py')

## use append to add path to google colab python path
sys.path.append(tbd_path)
#from module_net import load_tbd_net

utils_path = os.path.join(code_path, 'utils')
sys.path.append(utils_path)
from clevr import load_vocab

Mounted at /content/drive


# Extract Test Features

(only perform once)

In [ ]:
Test features

In [ ]:
def build_model():
  if not hasattr(torchvision.models, 'resnet101'):
    raise ValueError('Invalid model "%s"' % 'resnet101')
  if not 'resnet' in 'resnet101':
    raise ValueError('Feature extraction only supports ResNets')
  cnn = getattr(torchvision.models, 'resnet101')(pretrained=True)
  layers = [
    cnn.conv1,
    cnn.bn1,
    cnn.relu,
    cnn.maxpool,
  ]
  for i in range(3):
    name = 'layer%d' % (i + 1)
    layers.append(getattr(cnn, name))
  model = torch.nn.Sequential(*layers)
  model.cuda()
  model.eval()
  return model


def run_batch(cur_batch, model):
  mean = np.array([0.485, 0.456, 0.406]).reshape(1, 3, 1, 1)
  std = np.array([0.229, 0.224, 0.224]).reshape(1, 3, 1, 1)

  image_batch = np.concatenate(cur_batch, 0).astype(np.float32)
  image_batch = (image_batch / 255.0 - mean) / std
  image_batch = torch.FloatTensor(image_batch).cuda()
  image_batch = torch.autograd.Variable(image_batch, volatile=True)

  feats = model(image_batch)
  feats = feats.data.cpu().clone().numpy()

  return feats

input_image_dir = os.path.join(dataset_path,"raw/images/test")
max_images = None
image_height = 224
image_width = 224
output_h5_file = os.path.join(data_path,"test_features.h5")
batch_size = 64

input_paths = []
idx_set = set()
for fn in os.listdir(input_image_dir):
  if not fn.endswith('.png'): continue
  try:
    idx = int(os.path.splitext(fn)[0].split('_')[-1])
    input_paths.append((os.path.join(input_image_dir, fn), idx))
    idx_set.add(idx)
  except:
    print('invalid literal in int')    
input_paths.sort(key=lambda x: x[1])
assert len(idx_set) == len(input_paths)
assert min(idx_set) == 0 and max(idx_set) == len(idx_set) - 1
if max_images is not None:
  input_paths = input_paths[:max_images]
print(input_paths[0])
print(input_paths[-1])

model = build_model()

img_size = (image_height, image_width)
with h5py.File(output_h5_file, 'w') as f:
  feat_dset = None
  i0 = 0
  cur_batch = []
  for i, (path, idx) in enumerate(input_paths):
    img = imread(path, mode='RGB')
    img = imresize(img, img_size, interp='bicubic')
    img = img.transpose(2, 0, 1)[None]
    cur_batch.append(img)
    if len(cur_batch) == batch_size:
      feats = run_batch(cur_batch, model)
      if feat_dset is None:
        N = len(input_paths)
        _, C, H, W = feats.shape
        feat_dset = f.create_dataset('features', (N, C, H, W),
                                      dtype=np.float32)
      i1 = i0 + len(cur_batch)
      feat_dset[i0:i1] = feats
      i0 = i1
      print('Processed %d / %d images' % (i1, len(input_paths)))
      cur_batch = []
  if len(cur_batch) > 0:
    feats = run_batch(cur_batch, model)
    i1 = i0 + len(cur_batch)
    feat_dset[i0:i1] = feats
    print('Processed %d / %d images' % (i1, len(input_paths)))

Val features

In [ ]:
def build_model():
  if not hasattr(torchvision.models, 'resnet101'):
    raise ValueError('Invalid model "%s"' % 'resnet101')
  if not 'resnet' in 'resnet101':
    raise ValueError('Feature extraction only supports ResNets')
  cnn = getattr(torchvision.models, 'resnet101')(pretrained=True)
  layers = [
    cnn.conv1,
    cnn.bn1,
    cnn.relu,
    cnn.maxpool,
  ]
  for i in range(3):
    name = 'layer%d' % (i + 1)
    layers.append(getattr(cnn, name))
  model = torch.nn.Sequential(*layers)
  model.cuda()
  model.eval()
  return model


def run_batch(cur_batch, model):
  mean = np.array([0.485, 0.456, 0.406]).reshape(1, 3, 1, 1)
  std = np.array([0.229, 0.224, 0.224]).reshape(1, 3, 1, 1)

  image_batch = np.concatenate(cur_batch, 0).astype(np.float32)
  image_batch = (image_batch / 255.0 - mean) / std
  image_batch = torch.FloatTensor(image_batch).cuda()
  image_batch = torch.autograd.Variable(image_batch, volatile=True)

  feats = model(image_batch)
  feats = feats.data.cpu().clone().numpy()

  return feats

input_image_dir = os.path.join(dataset_path,"raw/images/val")
max_images = None
image_height = 224
image_width = 224
output_h5_file = os.path.join(data_path,"validate/val_features.h5")
batch_size = 64

input_paths = []
idx_set = set()
for fn in os.listdir(input_image_dir):
  if not fn.endswith('.png'): continue
  try:
    idx = int(os.path.splitext(fn)[0].split('_')[-1])
    input_paths.append((os.path.join(input_image_dir, fn), idx))
    idx_set.add(idx)
  except:
    print('invalid literal in int')    
input_paths.sort(key=lambda x: x[1])
assert len(idx_set) == len(input_paths)
assert min(idx_set) == 0 and max(idx_set) == len(idx_set) - 1
if max_images is not None:
  input_paths = input_paths[:max_images]
print(input_paths[0])
print(input_paths[-1])

model = build_model()

img_size = (image_height, image_width)
with h5py.File(output_h5_file, 'w') as f:
  feat_dset = None
  i0 = 0
  cur_batch = []
  for i, (path, idx) in enumerate(input_paths):
    img = imread(path, mode='RGB')
    img = imresize(img, img_size, interp='bicubic')
    img = img.transpose(2, 0, 1)[None]
    cur_batch.append(img)
    if len(cur_batch) == batch_size:
      feats = run_batch(cur_batch, model)
      if feat_dset is None:
        N = len(input_paths)
        _, C, H, W = feats.shape
        feat_dset = f.create_dataset('features', (N, C, H, W),
                                      dtype=np.float32)
      i1 = i0 + len(cur_batch)
      feat_dset[i0:i1] = feats
      i0 = i1
      print('Processed %d / %d images' % (i1, len(input_paths)))
      cur_batch = []
  if len(cur_batch) > 0:
    feats = run_batch(cur_batch, model)
    i1 = i0 + len(cur_batch)
    feat_dset[i0:i1] = feats
    print('Processed %d / %d images' % (i1, len(input_paths)))

('/content/drive/MyDrive/Project_NeurosymbolicAI/6.Datasets/CLEVR_v1.0/raw/images/val/CLEVR_val_000000.png', 0)
('/content/drive/MyDrive/Project_NeurosymbolicAI/6.Datasets/CLEVR_v1.0/raw/images/val/CLEVR_val_014999.png', 14999)


Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/checkpoints/resnet101-5d3b4d8f.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Processed 64 / 15000 images
Processed 128 / 15000 images
Processed 192 / 15000 images
Processed 256 / 15000 images
Processed 320 / 15000 images
Processed 384 / 15000 images
Processed 448 / 15000 images
Processed 512 / 15000 images
Processed 576 / 15000 images
Processed 640 / 15000 images
Processed 704 / 15000 images
Processed 768 / 15000 images
Processed 832 / 15000 images
Processed 896 / 15000 images
Processed 960 / 15000 images
Processed 1024 / 15000 images
Processed 1088 / 15000 images
Processed 1152 / 15000 images
Processed 1216 / 15000 images
Processed 1280 / 15000 images
Processed 1344 / 15000 images
Processed 1408 / 15000 images
Processed 1472 / 15000 images
Processed 1536 / 15000 images
Processed 1600 / 15000 images
Processed 1664 / 15000 images
Processed 1728 / 15000 images
Processed 1792 / 15000 images
Processed 1856 / 15000 images
Processed 1920 / 15000 images
Processed 1984 / 15000 images
Processed 2048 / 15000 images
Processed 2112 / 15000 images
Processed 2176 / 15000 ima

# Run Test Data through Model

Import libraries

In [4]:
from pathlib import Path
import numpy as np
import h5py

#from module_net import TbDNet

utils_path = os.path.join(code_path, 'utils')
sys.path.append(utils_path)
from clevr import load_vocab, ClevrDataLoaderH5
from generate_programs import load_program_generator, generate_programs


Set device

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Load vocab and trained model

In [9]:
from module_net import TbDNet

model_path = os.path.join(code_path, 'models')
vocab = load_vocab(Path(os.path.join(data_path,'vocab.json')))

tbd_net_checkpoint = os.path.join(model_path, 'example-15.pt')
tbd_net = TbDNet(vocab,feature_dim=(1024, 14, 14))
checkpoint = torch.load(tbd_net_checkpoint)
state_dict = checkpoint["state_dict"]
tbd_net.load_state_dict(state_dict)

if torch.cuda.is_available():
    tbd_net.cuda()

In [8]:
from module_net_back import TbDNet, load_tbd_net

model_path = os.path.join(code_path, 'models')
vocab = load_vocab(Path(os.path.join(data_path,'vocab.json')))

tbd_net_checkpoint = os.path.join(model_path, 'clevr.pt')
tbd_net = TbDNet(vocab,feature_dim=(1024, 14, 14))
tbd_net = load_tbd_net(tbd_net_checkpoint, vocab)
tbd_net.to(device)
#checkpoint = torch.load(tbd_net_checkpoint)
#state_dict = checkpoint["state_dict"]
#tbd_net.load_state_dict(state_dict)

#if torch.cuda.is_available():
    #tbd_net.cuda()

{'question_token_to_idx': {'<NULL>': 0, '<START>': 1, '<END>': 2, '<UNK>': 3, '': 4, ';': 5, 'Are': 6, 'Do': 7, 'Does': 8, 'How': 9, 'Is': 10, 'The': 11, 'There': 12, 'What': 13, 'a': 14, 'an': 15, 'and': 16, 'another': 17, 'any': 18, 'anything': 19, 'are': 20, 'as': 21, 'ball': 22, 'balls': 23, 'behind': 24, 'big': 25, 'block': 26, 'blocks': 27, 'blue': 28, 'both': 29, 'brown': 30, 'color': 31, 'cube': 32, 'cubes': 33, 'cyan': 34, 'cylinder': 35, 'cylinders': 36, 'does': 37, 'either': 38, 'else': 39, 'equal': 40, 'fewer': 41, 'front': 42, 'gray': 43, 'greater': 44, 'green': 45, 'has': 46, 'have': 47, 'how': 48, 'in': 49, 'is': 50, 'it': 51, 'its': 52, 'large': 53, 'left': 54, 'less': 55, 'made': 56, 'many': 57, 'material': 58, 'matte': 59, 'metal': 60, 'metallic': 61, 'more': 62, 'number': 63, 'object': 64, 'objects': 65, 'of': 66, 'on': 67, 'or': 68, 'other': 69, 'purple': 70, 'red': 71, 'right': 72, 'rubber': 73, 'same': 74, 'shape': 75, 'shiny': 76, 'side': 77, 'size': 78, 'small':

KeyboardInterrupt: ignored

Generate programs from questions (only perform once)

In [ ]:
#program_generator = load_program_generator(Path('models/program_generator.pt'))
program_generator = load_program_generator(os.path.join(model_path, 'program_generator.pt'))

#generate_programs(Path('data/test/test_questions.h5'), program_generator, 
                  #dest_dir=Path('data/test/'), batch_size=64)

generate_programs(os.path.join(data_path, 'test/test_questions.h5'), program_generator, 
                  dest_dir=os.path.join(data_path,'test/'), batch_size=64)

In [ ]:
#program_generator = load_program_generator(Path('models/program_generator.pt'))
program_generator = load_program_generator(os.path.join(model_path, 'program_generator.pt'))

#generate_programs(Path('data/test/test_questions.h5'), program_generator, 
                  #dest_dir=Path('data/test/'), batch_size=64)

generate_programs(os.path.join(data_path, 'validate/val_questions.h5'), program_generator, 
                  dest_dir=os.path.join(data_path,'validate/'), batch_size=64)

Generating programs...
Saved programs as /content/drive/MyDrive/Project_NeurosymbolicAI/7.Code/tbd-nets-encoder-main/data/validate/programs.npy
Saved image indices as /content/drive/MyDrive/Project_NeurosymbolicAI/7.Code/tbd-nets-encoder-main/data/validate/image_idxs.npy
Saved questions as /content/drive/MyDrive/Project_NeurosymbolicAI/7.Code/tbd-nets-encoder-main/data/validate/questions.npy


Load extracted image features, in addition to the questions, image indexes and programs we've just generated

In [7]:
#use_np_features = True
#if use_np_features:
    #features = np.load(str(Path('data/test/test_features.npy')), mmap_mode='r')
    #features = np.load(os.path.join(data_path,'test/test_features.npy'), mmap_mode='r')
#else:
    #features = h5py.File(Path('data/test/test_features.h5'))['features']
features = h5py.File(os.path.join(data_path,'test/test_features.h5'))['features']

#question_np = np.load(Path('data/test/test_questions.npy'))
#image_idx_np = np.load(Path('data/test/test_image_idxs.npy'))
#programs_np = np.load(Path('data/test/test_programs.npy'))

#question_np = np.load(os.path.join(data_path,'test/test_questions.h5'))
image_idx_np = np.load(os.path.join(data_path, 'test/image_idxs.npy'))
programs_np = np.load(os.path.join(data_path, 'test/programs.npy'))

Model output to answer mapping

In [17]:
answers_list = ['blue', 'brown', 'cyan', 'gray', 'green', 'purple', 'red', 'yellow',
           'cube', 'cylinder', 'sphere',
           'large', 'small',
           'metal', 'rubber',
           'no', 'yes',
           '0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9']

pred_idx_to_token = dict(zip(range(len(answers_list)), answers_list))

In [10]:
def map_ans(answers):
    ''' Map the answers from the format Justin Johnson et al. use to our own format. '''
    ans_tensor = torch.LongTensor(answers.size())
    ans_tensor[answers == 15] = 0  # blue
    ans_tensor[answers == 16] = 1  # brown
    ans_tensor[answers == 18] = 2  # cyan
    ans_tensor[answers == 20] = 3  # gray
    ans_tensor[answers == 21] = 4  # green
    ans_tensor[answers == 25] = 5  # purple
    ans_tensor[answers == 26] = 6  # red
    ans_tensor[answers == 30] = 7  # yellow
    ans_tensor[answers == 17] = 8  # cube
    ans_tensor[answers == 19] = 9  # cylinder
    ans_tensor[answers == 29] = 10 # sphere
    ans_tensor[answers == 22] = 11 # large
    ans_tensor[answers == 28] = 12 # small
    ans_tensor[answers == 23] = 13 # metal
    ans_tensor[answers == 27] = 14 # rubber
    ans_tensor[answers == 24] = 15 # no
    ans_tensor[answers == 31] = 16 # yes
    ans_tensor[answers == 4] = 17  # 0
    ans_tensor[answers == 5] = 18  # 1
    ans_tensor[answers == 6] = 19  # 10 <- originally sorted by ASCII value, not numerical
    ans_tensor[answers == 7] = 20  # 2
    ans_tensor[answers == 8] = 21  # 3
    ans_tensor[answers == 9] = 22  # 4
    ans_tensor[answers == 10] = 23 # 5
    ans_tensor[answers == 11] = 24 # 6
    ans_tensor[answers == 12] = 25 # 7
    ans_tensor[answers == 13] = 26 # 8
    ans_tensor[answers == 14] = 27 # 9
    return ans_tensor

Count ans_tensor = 17 - 27; Compare Numbers; Exist = 15 -16; Query Attribute; Compare Attribute


Check Program Type

In [11]:
def programType():
  batch_size = 64
  program_type = []
  for batch in range(0, len(programs_np)-batch_size, batch_size):
    image_idx = image_idx_np[batch:batch+batch_size]
    programs = torch.LongTensor(programs_np[batch:batch+batch_size]).to(device)
    print('batch: ',batch)
    for n in range(batch_size):
      program = []
      for i in reversed(programs.data[n].cpu().numpy()):  
        token = vocab['program_idx_to_token'][i]
        if token in {'<NULL>', '<START>', '<END>', '<UNK>', 'unique'}:
          continue
        else:
          program.append(token)
      print('\nprogram: ')
      print(program)
      'equal_size' (Size × Size → Boolean)
– equal material (Material × Material → Boolean)
– equal color (Color × Color → Boolean)
– equal shape (Shape × Shape → Boolean)
      if any("relate" in x for x in program):
        #print('**********************compare type')
        program_type.append('compare')
      elif any(x in {'equal', 'intersect', 'union', 'less_than','greater_than'} for x in program):
        #print('**********************compare int type')
        program_type.append('compare_int')
      elif any(x in {'query_material','query_size','query_color','query_shape'} for x in program):
        #print('**********************query type')
        program_type.append('query')
      elif any("exist" in x for x in program):
        #print('**********************exist type')
        program_type.append('exist')
      elif any("count" in x for x in program):
        #print('**********************count type')
        program_type.append('count')
  return program_type

SyntaxError: ignored

In [ ]:
program_type = programType()
print(len(program_type))

Check Accuracy

In [12]:
val_loader_kwargs = {

    'question_h5': os.path.join(data_path,'validate/val_questions.h5'),
    'feature_h5': os.path.join(data_path,'validate/val_features.h5'),
    'batch_size': 64,
    'num_workers': 2,
    'shuffle': False
}
val_loader = ClevrDataLoaderH5(**val_loader_kwargs)

Reading features from  /content/drive/MyDrive/Project_NeurosymbolicAI/7.Code/tbd-nets-encoder-main/data/validate/val_features.h5


In [ ]:
def check_accuracy(loader):
    '''
    Convenience function for checking the accuracy of the model 
    over all the data in a given `DataLoader`
    '''
    compare_total = 0
    compare_correct = 0
    compare_int_total = 0
    compare_int_correct = 0
    query_total = 0
    query_correct = 0
    exist_total = 0
    exit_correct = 0
    count_total = 0
    count_correct = 0
    batch_size = 64
    torch.set_grad_enabled(False)
    num_correct, num_samples = 0, 0
    batch_no = 0 
    for batch in loader:
        _, _, feats, answers, programs = batch
        feats = feats.to(device)
        programs = programs.to(device)
        outs = tbd_net(feats, programs)
        print("batch_no = ", batch_no)
        batch_no = batch_no+1
        program_type = []
        if len(programs) == batch_size:
          for n in range(batch_size):
            program = []
            for id in reversed(programs.data[n].cpu().numpy()):  
              #print('id',id)
              token = vocab['program_idx_to_token'][id]
              if token in {'<NULL>', '<START>', '<END>', '<UNK>', 'unique'}:
                continue
              else:
                program.append(token)
            print('\nn:',n)    
            print('program: ')
            print(program)
            print('program[len(program)] : ')
  
            if any("relate" in x for x in program):
              program_type.append('compare')
            elif any(x in {'equal', 'intersect', 'union', 'less_than','greater_than'} for x in program):
              program_type.append('compare_int')
            elif any(x in {'query_material','query_size','query_color','query_shape'} for x in program):
              program_type.append('query')
            elif any("exist" in x for x in program):
              program_type.append('exist')
            elif any("count" in x for x in program):
              program_type.append('count')

          _, preds = outs.max(1)
          mapped_ans = map_ans(answers)
          num_correct += (preds.to('cpu') == mapped_ans).sum()
          num_samples += preds.size(0)
          
          correct=[]
          #print(programs)
          preds_str = [pred_idx_to_token[pred] for pred in preds.detach().to('cpu').numpy()]
          #print('preds_str: ') 
          #print(preds_str) 
          predictions = preds.cpu().detach().numpy()
          mapped_answers = mapped_ans.cpu().detach().numpy()
          count = 0
          for pred in predictions:
            correct.append(pred == mapped_answers[count])
            #print('prediction: ')
            #print(pred)
            #print('answer: ')          
            #print(mapped_answers[count])          
            count = count + 1                  
            
          for i in range (0,batch_size):
            category = program_type[i]
            if category == 'compare':
              #print('************************************************************** compare')
              #print(preds_str[i])
              compare_total = compare_total + 1
              if correct[i] == True:
                #print('************************************************************** compare correct')
                compare_correct = compare_correct + 1
            if category == 'compare_int':
              #print('************************************************************** compare int')
              #print(preds_str[i])
              compare_int_total = compare_int_total + 1
              if correct[i] == True:
                #print('************************************************************** compare int correct')
                compare_int_correct = compare_int_correct + 1
            if category == 'query':
              #print('************************************************************** query')
              #print(preds_str[i])
              query_total = query_total + 1
              if correct[i] == True:
                #print('************************************************************** query correct')
                query_correct = query_correct + 1
            if category == 'exist':
              #print('************************************************************** exist')
              #print(preds_str[i])
              exist_total = exist_total + 1
              if correct[i] == True:
                #print('************************************************************** exit correct')
                exit_correct = exit_correct + 1
            if category == 'count':
              #print('************************************************************** count')
              #print(preds_str[i])
              count_total = count_total + 1
              if correct[i] == True:
                #print('************************************************************** count correct')
                count_correct = count_correct + 1

    print('compare_total: ', compare_total)
    print('compare_correct: ', compare_correct)
    print('% correct: ', compare_correct*100/compare_total)

    print('compare_int_total: ', compare_int_total)
    print('compare_int_correct: ', compare_int_correct)
    print('% correct: ', compare_int_correct*100/compare_int_total)

    print('query_total: ', query_total)
    print('query_correct: ', query_correct)
    print('% correct: ', query_correct*100/query_total)

    print('exist_total: ', exist_total)
    print('exit_correct: ', exit_correct)
    print('% correct: ', exit_correct*100/exist_total)

    print('count_total: ', count_total)
    print('count_correct: ', count_correct)
    print('% correct: ', count_correct*100/count_total)

    return (num_correct, num_samples)

check_accuracy(val_loader)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


Streaming output truncated to the last 5000 lines.
n: 30
program: 
['scene', 'filter_size[large]', 'filter_color[gray]', 'scene', 'filter_size[small]', 'filter_color[blue]', 'union', 'filter_material[rubber]', 'count']
program[len(program)] : 

n: 31
program: 
['scene', 'filter_size[large]', 'filter_shape[cylinder]', 'relate[behind]', 'query_shape', 'scene', 'filter_color[cyan]', 'filter_shape[cube]', 'relate[front]', 'filter_size[small]', 'query_shape', 'equal_shape']
program[len(program)] : 

n: 32
program: 
['scene', 'filter_size[small]', 'filter_color[cyan]', 'relate[left]', 'scene', 'filter_size[small]', 'filter_color[purple]', 'filter_material[metal]', 'filter_shape[sphere]', 'relate[front]', 'intersect', 'filter_size[small]', 'query_color']
program[len(program)] : 

n: 33
program: 
['scene', 'filter_size[large]', 'filter_shape[sphere]', 'same_color', 'filter_shape[sphere]', 'query_size']
program[len(program)] : 

n: 34
program: 
['scene', 'filter_size[large]', 'filter_color[brow

(tensor(147990), 149952)

# program size test

In [22]:
def program_sizes(loader):
    '''
    Convenience function for checking the accuracy of the model 
    over all the data in a given `DataLoader`
    '''
    _1_total = 0
    _1_correct = 0
    _2_total = 0
    _2_correct = 0
    _3_total = 0
    _3_correct = 0
    _4_total = 0
    _4_correct = 0
    _5_total = 0
    _5_correct = 0
    _6_total = 0
    _6_correct = 0
    _7_total = 0
    _7_correct = 0
    _8_total = 0
    _8_correct = 0
    _9_total = 0
    _9_correct = 0
    _10_total = 0
    _10_correct = 0
    _11_total = 0
    _11_correct = 0
    _12_total = 0
    _12_correct = 0
    _13_total = 0
    _13_correct = 0
    _14_total = 0
    _14_correct = 0
    _15_total = 0
    _15_correct = 0
    _16_total = 0
    _16_correct = 0
    _17_total = 0
    _17_correct = 0
    _18_total = 0
    _18_correct = 0



    batch_size = 64
    torch.set_grad_enabled(False)
    num_correct, num_samples = 0, 0
    batch_no = 0 
    for batch in loader:
        _, _, feats, answers, programs = batch
        feats = feats.to(device)
        programs = programs.to(device)
        outs = tbd_net(feats, programs)
        print("batch_no = ", batch_no)
        batch_no = batch_no+1
        program_length = []
        if len(programs) == batch_size:
          for n in range(batch_size):
            program = []
            for id in reversed(programs.data[n].cpu().numpy()):  
              #print('id',id)
              token = vocab['program_idx_to_token'][id]
              if token in {'<NULL>', '<START>', '<END>', '<UNK>', 'unique'}:
                continue
              else:
                program.append(token)
            print('\nn:',n)    
            print('program: ')
            print(program)
           

            
            program_length.append(len(program))
            len(program)
            print('program[len(program)] : ', len(program))

          _, preds = outs.max(1)
          mapped_ans = map_ans(answers)
          num_correct += (preds.to('cpu') == mapped_ans).sum()
          num_samples += preds.size(0)
          
          correct=[]
          #print(programs)
          preds_str = [pred_idx_to_token[pred] for pred in preds.detach().to('cpu').numpy()]
          #print('preds_str: ') 
          #print(preds_str) 
          predictions = preds.cpu().detach().numpy()
          mapped_answers = mapped_ans.cpu().detach().numpy()
          count = 0
          for pred in predictions:
            correct.append(pred == mapped_answers[count])
            #print('prediction: ')
            #print(pred)
            #print('answer: ')          
            #print(mapped_answers[count])          
            count = count + 1                  
            
          for i in range (0,batch_size):
            pro_len = program_length[i]
            if pro_len == 1:
              _1_total = _1_total + 1
              if correct[i] == True:
                _1_correct = _1_correct + 1
            if pro_len == 2:
              _2_total = _2_total + 1
              if correct[i] == True:
                _2_correct = _2_correct + 1
            if pro_len == 3:
              _3_total = _3_total + 1
              if correct[i] == True:
                _3_correct = _3_correct + 1
            if pro_len == 4:
              _4_total = _4_total + 1
              if correct[i] == True:
                _4_correct = _4_correct + 1
            if pro_len == 5:
              _5_total = _5_total + 1
              if correct[i] == True:
                _5_correct = _5_correct + 1            
            if pro_len == 6:
              _6_total = _6_total + 1
              if correct[i] == True:
                _6_correct = _6_correct + 1
            if pro_len == 7:
              _7_total = _7_total + 1
              if correct[i] == True:
                _7_correct = _7_correct + 1
            if pro_len == 8:
              _8_total = _8_total + 1
              if correct[i] == True:
                _8_correct = _8_correct + 1            
            if pro_len == 9:
              _9_total = _9_total + 1
              if correct[i] == True:
                _9_correct = _9_correct + 1
            if pro_len == 10:
              _10_total = _10_total + 1
              if correct[i] == True:
                _10_correct = _10_correct + 1
            if pro_len == 11:
              _11_total = _11_total + 1
              if correct[i] == True:
                _11_correct = _11_correct + 1
            if pro_len == 12:
              _12_total = _12_total + 1
              if correct[i] == True:
                _12_correct = _12_correct + 1
            if pro_len == 13:
              _13_total = _13_total + 1
              if correct[i] == True:
                _13_correct = _13_correct + 1
            if pro_len == 14:
              _14_total = _14_total + 1
              if correct[i] == True:
                _14_correct = _14_correct + 1
            if pro_len == 14:
              _14_total = _14_total + 1
              if correct[i] == True:
                _14_correct = _14_correct + 1
            if pro_len == 15:
              _15_total = _15_total + 1
              if correct[i] == True:
                _15_correct = _15_correct + 1
            if pro_len == 16:
              _16_total = _16_total + 1
              if correct[i] == True:
                _16_correct = _16_correct + 1
            if pro_len == 17:
              _17_total = _17_total + 1
              if correct[i] == True:
                _17_correct = _17_correct + 1
            if pro_len == 18:
              _18_total = _18_total + 1
              if correct[i] == True:
                _18_correct = _18_correct + 1


    print('1_total: ', _1_total)
    print('1_correct: ', _1_correct)
    try:
      print('% 1_correct: ', _1_correct*100/_1_total)
    except:
      print('0%')
    print('2_total: ', _2_total)
    print('2_correct: ', _2_correct)
    try:
      print('% 2_correct: ', _2_correct*100/_2_total)
    except:
      print('0%')

    print('3_total: ', _3_total)
    print('3_correct: ', _3_correct)
    try:
      print('% 3_correct: ', _3_correct*100/_3_total)
    except:
      print('0%')

    print('4_total: ', _4_total)
    print('4_correct: ', _4_correct)
    try:
      print('% 4_correct: ', _4_correct*100/_4_total)
    except:
      print('0%')
    print('5_total: ', _5_total)
    print('5_correct: ', _5_correct)
    try:
      print('% 5_correct: ', _5_correct*100/_5_total)
    except:
      print('0%')

    print('6_total: ', _6_total)
    print('6_correct: ', _6_correct)
    try:
      print('% 6_correct: ', _6_correct*100/_6_total)
    except:
      print('0%')

    print('7_total: ', _7_total)
    print('7_correct: ', _7_correct)
    try:
      print('% 7_correct: ', _7_correct*100/_7_total)
    except:
      print('0%')

    print('8_total: ', _8_total)
    print('8_correct: ', _8_correct)
    try:
      print('% 8_correct: ', _8_correct*100/_8_total)
    except:
      print('0%')

    print('9_total: ', _9_total)
    print('9_correct: ', _9_correct)
    try:  
      print('% 9_correct: ', _9_correct*100/_9_total)
    except:
      print('0%')

    print('10_total: ', _10_total)
    print('10_correct: ', _10_correct)
    try:
      print('% 10_correct: ', _10_correct*100/_10_total)
    except:
      print('0%')

    print('11_total: ', _11_total)
    print('11_correct: ', _11_correct)
    try:
      print('% 11_correct: ', _11_correct*100/_11_total)
    except:
      print('0%')

    print('12_total: ', _12_total)
    print('12_correct: ', _12_correct)
    try:
      print('% 12_correct: ', _12_correct*100/_12_total)
    except:
      print('0%')

    print('13_total: ', _13_total)
    print('13_correct: ', _13_correct)
    try:
      print('% 13_correct: ', _13_correct*100/_13_total)
    except:
      print('0%')

    print('14_total: ', _14_total)
    print('14_correct: ', _14_correct)
    try:
      print('% 14_correct: ', _14_correct*100/_14_total)
    except:
      print('0%')

    print('15_total: ', _15_total)
    print('15_correct: ', _15_correct)
    try:
      print('% 15_correct: ', _15_correct*100/_15_total)
    except:
      print('0%')

    print('16_total: ', _16_total)
    print('16_correct: ', _16_correct)
    try:
      print('% 16_correct: ', _16_correct*100/_16_total)
    except:
      print('0%')

    print('17_total: ', _17_total)
    print('17_correct: ', _17_correct)
    try:
      print('% 17_correct: ', _17_correct*100/_17_total)
    except:
      print('0%')

    print('18_total: ', _18_total)
    print('18_correct: ', _18_correct)
    try:
      print('% 18_correct: ', _18_correct*100/_18_total)
    except:
      print('0%')

program_sizes(val_loader)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Streaming output truncated to the last 5000 lines.

n: 38
program: 
['scene', 'filter_size[small]', 'filter_color[gray]', 'filter_shape[sphere]', 'relate[right]', 'filter_size[small]', 'filter_color[gray]', 'filter_material[metal]', 'relate[right]', 'filter_shape[cube]', 'query_material']
program[len(program)] :  11

n: 39
program: 
['scene', 'filter_color[green]', 'query_shape']
program[len(program)] :  3

n: 40
program: 
['scene', 'filter_size[large]', 'filter_color[gray]', 'filter_material[rubber]', 'relate[left]', 'filter_size[large]', 'filter_material[metal]', 'count', 'scene', 'filter_size[small]', 'filter_shape[sphere]', 'relate[behind]', 'filter_material[metal]', 'filter_shape[sphere]', 'count', 'equal_integer']
program[len(program)] :  16

n: 41
program: 
['scene', 'filter_size[small]', 'filter_color[green]', 'relate[left]', 'filter_size[small]', 'query_color', 'scene', 'filter_color[gray]', 'filter_material[metal]', 'filter_shape[sphere]', 'relate[behind]', 'filter_material[r

Run through all of the questions and write output prediction to file

In [11]:
compare_total = 0
compare_correct = 0

compare_int_total = 0
compare_int_correct = 0

query_total = 0
query_correct = 0

exist_total = 0
exit_correct = 0

count_total = 0
count_correct = 0

batch_size = 64
program_count = 0
for batch in range(0, len(programs_np), batch_size):
    image_idx = image_idx_np[batch:batch+batch_size]
    programs = torch.LongTensor(programs_np[batch:batch+batch_size]).to(device)
    
    #if use_np_features:
    #    feats = torch.FloatTensor(np.asarray(features[image_idx])).to(device)
    #else:
    # Using HDF5 files requires some overhead due to constraints on how those may
    # be accessed. We cannot index into the file using a numpy array. We also cannot 
    # access the same element multiple times (e.g. we cannot index into an h5py.File 
    # with [1,1,1]) because we are constrained to increasing sequences
    feats = []
    for idx in image_idx:
        feats.append(np.asarray(features[idx]))
    feats = torch.FloatTensor(np.asarray(feats)).to(device)

    outputs = tbd_net(feats, programs)
    _, preds = outputs.max(1)
    mapped_ans = map_ans(answers)

    category = program_type[program_count]
    program_count = program_count + 1
    correct = preds.to('cpu') == mapped_ans
    if program_type == 'compare':
      compare_total = compare_total + 1
      if correct:
        compare_correct = compare_correct + 1
    if program_type == 'compare_int':
      compare_int_total = compare_int_total + 1
      if correct:
        compare_int_correct = compare_int_correct + 1
    if program_type == 'query':
      query_total = query_total + 1
      if correct:
        query_correct = query_correct + 1
    if program_type == 'exist':
      exist_total = exist_total + 1
      if correct:
        exit_correct = exit_correct + 1
    if program_type == 'count':
      count_total = count_total + 1
      if correct:
        count_correct = count_correct + 1

    #num_correct += (preds.to('cpu') == mapped_ans).sum()
    #num_samples += preds.size(0)

    #preds = [pred_idx_to_token[pred] for pred in preds.detach().to('cpu').numpy()]
    #write_preds(preds)
#f.close()
print('compare_total: ', compare_total)
print('compare_correct: ', compare_correct)
print('% correct: ', compare_correct*100/compare_total)

print('compare_int_total: ', compare_int_total)
print('compare_int_correct: ', compare_int_correct)
print('% correct: ', compare_int_correct*100/compare_total)

print('query_total: ', query_total)
print('query_correct: ', query_correct)
print('% correct: ', query_correct*100/compare_total)

print('exist_total: ', exist_total)
print('exit_correct: ', exit_correct)
print('% correct: ', exit_correct*100/compare_total)

print('count_total: ', count_total)
print('count_correct: ', count_correct)
print('% correct: ', count_correct*100/compare_total)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


AttributeError: ignored